# I. Data extraction

### Methodology

**Table used are:**

* VitalPeriodic: include vital information of the patient, such as temperature, heart rate, respiration, blood pressure.... from bedside vital signs monitors and was aggregated as 5 minutes interval. This table include majority of the value required from the research papers, while other table such as nursecharting having data manually input so it is fairly unstructure.
* Lab: include lab results, such as albumin, bun, bilirubin, lactate, bicarbonate, ... but the lab result all stored in 1 columns, the type of the test are in column 'labname' and the value of the test are in column 'labresult'.
* Patient: include patient demographic information (age, gender, ethnicity) and time admitted to ICU.
 

I created 8 different CTE in one query, all connected by the patientunitstayID. Below is a detailed explanation of each CTE:

1. **`upd_vitalperiodic`**:
    - **Purpose**: Cleans and processes the vital signs data.
    - **Details**:
        - Selects vital signs from the `vitalperiodic` table.
        - Ensures the values of vital signs fall within valid ranges by setting values outside these ranges to `NULL`.
        - If there are missing value, then uses the `LAST_VALUE` window function to carry forward the last non-`NULL` value for each vital sign within each `patientunitstayid` partition.

2. **`vt`**:
    - **Purpose**: Aggregates the processed vital signs data into hourly intervals.
    - **Details**:
        - Groups the `upd_vitalperiodic` data into 1-hour intervals (each interval represents 60 minutes).
        - Calculates the median value of each vital sign within each interval.
        - Joins the `patient` table to filter records based on `observationoffset` constraints (Take only the Vital recorded before patient discarded, as vital may include multiple visits).

3. **`upd_laboratory`**:
    - **Purpose**: Cleans and processes the laboratory test results.
    - **Details**:
        - Selects laboratory test results from the `lab` table.
        - Ensures the lab results fall within valid ranges by setting values outside these ranges to `NULL`.
        - If there are missing value, uses the `LAST_VALUE` window function to carry forward the last non-`NULL` value for each lab result within each `patientunitstayid` partition.

4. **`laboratory`**:
    - **Purpose**: Aggregates the processed laboratory test results into 8-hour intervals.
    - **Details**:
        - Groups the `upd_laboratory` data into 8-hour intervals (each interval represents 480 minutes).
        - Calculates the median value of each lab result within each interval.
        - Joins the `patient` table to filter records based on `labResultOffset` constraints (Take Lab result taken before patient discarded, as lab may measured in multiple visits).

5. **`condition_vt`**:
    - **Purpose**: Calculates the minimum and maximum `observationoffset` values for each `patientunitstayid`.

6. **`condition_lab`**:
    - **Purpose**: Calculates the minimum and maximum `labResultOffset` values for each `patientunitstayid`.

7. **`vt_lab`**:
    - **Purpose**: Identifies patients who have both vital sign and lab result observations longer than 48 hours.
    - **Details**:
        - Joins `condition_vt` and `condition_lab` to find patients whose observation periods for both vital signs and lab results exceed 48 hours (2880 minutes).

8. **`final_patient`**:
    - **Purpose**: Selects the first visit for each unique patient and ensures the visit duration is at least 48 hours.
    - **Details**:
        - Uses a window function (`ROW_NUMBER`) to identify the first visit for each unique patient.
        - Joins the result with `vt_lab` to include only patients with long observation periods.
        - Filters to include only visits with ICU duration longer than 48 hours (2880 minutes).

**Final Query**: I pulled 3 separate dataset for further analysis

1. **`Lab result`**
- **Purpose**: Combines all processed and aggregated data to produce the final lab result set.
- **Details**:
    - Joins `laboratory` and `final_patient` on `patientunitstayid`.
    - Selects all the laboratory result columns such as bilirubin, BUN....
    - Filters the intervals to include only those within 24 hours before discharge.
    - Orders the final result set by `patientunitstayid` and `interval`.
 
2. **`Vital sign`**
- **Purpose**: Combines all processed and aggregated data to produce the final vital signs set.
- **Details**:
    - Joins `vitalperiodic` and `final_patient` on `patientunitstayid`.
    - Selects all the laboratory result columns such as heart rate, systolic, diastolic,...
    - Filters the intervals to include only those within 24 hours before discharge.
    - Orders the final result set by `patientunitstayid` and `interval`.

3. **`Patient demographics`**
- **Purpose**: Get patient infomation: gender, age, ethnicity from final_patient CTE.


I got the same number with the paper after filter the patient with the first ICU admission (139,367 unique patient). However, after set up condition about the ICU duration, vital, lab duration, I got final data with 44,909 observations compared with 36,283.

### Flow chart

**Figure 1: Query structure**
<img src="data/andy.png" width=900 height=600 />

**Figure 2: Conditions used in the research paper**
<img src="data/condition.png" width=700 height=400 />
